# Introduction to Memory Profiling

> Objectives:
> * Be introduced to memory profiling using different tools
> * Some small introduction to time profiling in IPython too


## ipython_memwatcher

Our recommended way to profile memory consumption for this tutorial will be [ipython_memwatcher](https://pypi.python.org/pypi/ipython_memwatcher):


In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.020 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 30.293 MiB


In [2]:
# Let's create a big object
a = [i for i in range(1000*1000)]

In [2] used 31.203 MiB RAM in 0.066s, peaked 0.000 MiB above current, total RAM usage 61.496 MiB


In [3]:
# Get some measurements from the last executed cell:
meas = mw.measurements
meas

Measurements(memory_delta=31.203125, time_delta=0.06564903259277344, memory_peak=0, memory_usage=61.49609375)

In [3] used 0.062 MiB RAM in 0.017s, peaked 0.000 MiB above current, total RAM usage 61.559 MiB


In [4]:
# MemWatcher.measurements is a named tuple.  We can easily get info out of it:
meas.memory_delta

31.203125

In [4] used 0.012 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 61.570 MiB


In [5]:
# This takes between 32 ~ 35 bytes per element:
meas.memory_delta * (2**20) / 1e6

32.718848

In [5] used 0.000 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 61.570 MiB


In [6]:
# What are these elements made from?
type(a[0])

int

In [6] used 0.008 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 61.578 MiB


In [7]:
# How much memory take an int?
# Beware: the size below will depend on whether you are using a 32-bit or 64-bit Python
import sys
sys.getsizeof(a[0])

24

In [7] used 0.008 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 61.586 MiB


Ok.  On 64-bits, that means that the int object allocates 8 bytes for the integer value, and 16 bytes for other metadata (Python object).  But 24 is quite less than 32~35.  Where this overhead comes from?

Well, it turns out that the list structure needs additional pointers to reference the different components. [Explain with some diagrams]

## memory_profiler

[memory_profiler](https://pypi.python.org/pypi/memory_profiler) is a basic module for memory profiling that many others use (like the `ipython_memwatcher` above) and it interacts well with ipython, so it is worth to see how it works:

In [8]:
%load_ext memory_profiler

In [8] used 0.020 MiB RAM in 0.005s, peaked 0.000 MiB above current, total RAM usage 61.605 MiB


In [9]:
# Use %memit magic command exposed by memory_profiler
%memit b = [i for i in range(1000*1000)]

peak memory: 131.30 MiB, increment: 69.69 MiB
In [9] used 69.852 MiB RAM in 0.290s, peaked 0.000 MiB above current, total RAM usage 131.457 MiB


Please note that the `peak_memory` in this case is different than the `peaked_memory` reported by ipython_memwatcher package.

## Guppy

Guppy is nice for having an overview of how different structures are using our memory:

In [10]:
from guppy import hpy; hp=hpy()
hp.heap()

Partition of a set of 2131793 objects. Total size = 82441016 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 2002850  94 48068400  58  48068400  58 int
     1   1276   0 16451680  20  64520080  78 list
     2  59738   3  5226152   6  69746232  85 str
     3  31923   1  2736616   3  72482848  88 tuple
     4   1799   0  1759016   2  74241864  90 dict (no owner)
     5    483   0  1370952   2  75612816  92 dict of module
     6   8200   0  1049600   1  76662416  93 types.CodeType
     7   8028   0   963360   1  77625776  94 function
     8    948   0   854144   1  78479920  95 type
     9    948   0   770784   1  79250704  96 dict of type
<584 more rows. Type e.g. '_.more' to view.>

In [10] used 33.320 MiB RAM in 2.868s, peaked 17.500 MiB above current, total RAM usage 164.777 MiB


In [11]:
del b
hp.heap()

Partition of a set of 1132069 objects. Total size = 50326800 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 1003108  89 24074592  48  24074592  48 int
     1   1273   0  8325000  17  32399592  64 list
     2  59743   5  5226472  10  37626064  75 str
     3  31924   3  2736720   5  40362784  80 tuple
     4   1805   0  1762232   4  42125016  84 dict (no owner)
     5    483   0  1370952   3  43495968  86 dict of module
     6   8200   1  1049600   2  44545568  89 types.CodeType
     7   8027   1   963240   2  45508808  90 function
     8    948   0   854144   2  46362952  92 type
     9    948   0   770784   2  47133736  94 dict of type
<580 more rows. Type e.g. '_.more' to view.>

In [11] used 34.801 MiB RAM in 0.598s, peaked 0.000 MiB above current, total RAM usage 199.578 MiB


In [12]:
# Size of the list (beware, this does not include the contents!)
hp.iso(a)

Partition of a set of 1 object. Total size = 8126536 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1 100  8126536 100   8126536 100 list

In [12] used 0.031 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 199.609 MiB


## %time and %timeit

In [13]:
# IPython provides a magic command to see how much time a command takes to run
%time asum = sum(a)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.3 ms
In [13] used 0.008 MiB RAM in 0.013s, peaked 0.000 MiB above current, total RAM usage 199.617 MiB


Note that `%time` offers quite detailed statistics on the time spent.

Also, the time reported by MemoryWatcher has a typical overhead of 1~5 ms over the time reported by %time, so when the times to measure are about this order then it is better to rely on the %time (or %timeit below) values.  

In [14]:
# We have another way to measure timings doing several loops and getting the mean
%timeit bsum = sum(a)
# However, one must notice that %timeit does not return the result of expressions

100 loops, best of 3: 6.18 ms per loop
In [14] used 0.016 MiB RAM in 2.578s, peaked 0.000 MiB above current, total RAM usage 199.633 MiB


Interestingly, %timeit allows to retrieve the measured times in loops with the -o flag:

In [15]:
t = %timeit -o sum(a)
print(t.all_runs)
print(t.best)

100 loops, best of 3: 6.19 ms per loop
[0.6193509101867676, 0.6191778182983398, 0.6193079948425293]
0.00619177818298
In [15] used 0.020 MiB RAM in 2.558s, peaked 0.000 MiB above current, total RAM usage 199.652 MiB


And one can specify the number of loops (-n) and the number of repetitions (-r):

In [16]:
t = %timeit -r1 -n1 -o sum(a)
print(t.all_runs)
print(t.best)

1 loops, best of 1: 9.54 ms per loop
[0.00953817367553711]
0.00953817367554
In [16] used 0.000 MiB RAM in 0.012s, peaked 0.000 MiB above current, total RAM usage 199.652 MiB


### Exercise 1

Provided a dictionary like:

```
d = dict(("key: %i"%i, i*2) for i in a)
```

Try to guess how much RAM it uses using the techniques introduced above.

Why do you think it takes more space than a list?

*Hint*: Every entry in a dictionary has pointers to two objects: key and value. 

## Solution

In [17]:
d = dict(("key: %i"%i, i*2) for i in a)

In [17] used 97.230 MiB RAM in 0.681s, peaked 0.000 MiB above current, total RAM usage 296.883 MiB


In [18]:
# Compute the size of key + value
sys.getsizeof("key: 100000") + sys.getsizeof(1)

72

In [18] used 0.004 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 296.887 MiB


In [20]:
# Compute the size of the dict struct (using guppy here)
hp.iso(d)

Partition of a set of 1 object. Total size = 50331928 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1 100 50331928 100  50331928 100 dict (no owner)

In [20] used 0.004 MiB RAM in 0.527s, peaked 0.000 MiB above current, total RAM usage 296.891 MiB


So, guppy is telling us that just the dictionary structure is taking ~50 MB, whereas the contents alone are taking ~70MB, so we should have expected the dictionary to consume ~120 MB.  However, our `MemWatcher` instance is reporting just ~100 MB.  Take away lesson: measuring memory consumption in Python is tricky but using proper tools we can still get valuable hints!